In [ ]:
import matplotlib.dates as mdates

max_time = 24 * 7 # change this if you want to plot for longer periods
time_range = range(0, max_time)

results_basecase_summer['timestamp'] = pd.to_datetime(results_basecase_summer['timestamp'])

timestamps = results_basecase_summer['timestamp'][:max_time]

data_power = pd.DataFrame({
    'timestamp': timestamps,
    'Inflexible Demand': [inflex_demand[t] for t in time_range],
    'Dishwasher': [power_dishwasher * binary_dishwasher[t] for t in time_range],
    'Washing Machine': [power_wm * binary_wm[t] for t in time_range],
    'Dryer': [power_dryer * binary_dryer[t] for t in time_range],
    'EV Charging': [charging_ev[t] for t in time_range]
})


#choose your own colors for the plot
custom_colors = {
    'Inflexible Demand': '#1f77b4',  # blue
    'Dishwasher': '#ff7f0e',         # orange
    'Washing Machine': '#2ca02c',    # green
    'Dryer': '#d62728',              # red
    'EV Charging': '#9467bd'         # purple
}

plt.rcParams.update({
    'axes.titlesize': 16,     # Title font
    'axes.labelsize': 11,     # x/y labels
    'legend.fontsize': 11,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11
})

palette = sns.color_palette("tab10", n_colors=5) # choose colorpalette from seaborn

data_power_melted = data_power.melt(id_vars='timestamp', var_name='Technology', value_name='Power (kWh)')

fig, ax1 = plt.subplots(figsize=(14, 7))

sns.lineplot(data=data_power_melted, x='timestamp', y='Power (kWh)', hue='Technology', ax=ax1, palette=palette)
ax1.set_ylabel("Power (kWh)")
ax1.set_xlabel("Day")

# Align x-axis to exact days
start_time = timestamps.iloc[0].normalize()
end_time = timestamps.iloc[-1].normalize() + pd.Timedelta(days=1)
ax1.set_xlim(start_time, end_time)

# Format tick labels: e.g., "Tue 01.08"
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%a %d.%m'))
ax1.xaxis.set_major_locator(mdates.DayLocator())

# Spotmarket price (right axis)
ax2 = ax1.twinx()
ax2.plot(timestamps, spot_price[:max_time], 'r--', label='Spotmarket Price')
ax2.set_ylabel("Spotmarket Price (EUR/kWh)", color='black')
ax2.tick_params(axis='y', labelcolor='black')

# EV SOC (third axis)
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))
ax3.plot(timestamps, soc_ev[:max_time] / 70, 'g:', label='EV SOC')
ax3.set_ylabel("EV SOC", color='black')
ax3.tick_params(axis='y', labelcolor='black')

# Legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()

legend = ax1.legend(
    lines1 + lines2 + lines3,
    labels1 + labels2 + labels3,
    loc='upper right',
    fontsize=9,
    labelspacing=0.3,
    borderpad=0.3,
    handlelength=1.2,
    handletextpad=0.4,
    borderaxespad=0.5, # No frame around the legend
    fancybox=True,  # Use a rounded box background
    facecolor='#f9f9f9',  # Background color for the legend
)

legend.get_frame().set_alpha(0.9)

plt.title("Power Demand, Spotmarket Price, and EV SOC Over One Week")
plt.tight_layout()
ax1.grid(False)         # Only show horizontal gridlines
ax2.grid(False)            # No grid on price axis
ax3.grid(False)
plt.savefig(os.path.join(figures_dir, "example_plot_profiles.png"), dpi=400, bbox_inches='tight')
plt.show() 